In [1]:
import os
import json
import transformers
from transformers import AutoConfig, AutoTokenizer, HfArgumentParser, set_seed
from tqdm import tqdm
from arguments import ModelArguments, DataTrainingArguments, GLMTrainingArguments
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from utils import *
from model import GLMModelforIND, LlamaModelForIND, Qwen2ModelForIND
from torch.nn import DataParallel
from dataset import INSTRUCTION, MAGDataset, RawDataset


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, GLMTrainingArguments))
model_args, data_args, training_args = parser.parse_json_file(
    json_file="configs/llama3/eval.json"
)

def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.
    From https://github.com/tatsu-lab/stanford_alpaca/blob/main/train.py

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(
            dim=0, keepdim=True
        )
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(
            dim=0, keepdim=True
        )

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg

set_seed(47)
config = AutoConfig.from_pretrained(
    model_args.model_name_or_path, trust_remote_code=True
)
config.use_cache = False
# config._attn_implementation = "flash_attention_2" #use flash attention
config.model_args = model_args
tokenizer = AutoTokenizer.from_pretrained(
    model_args.model_name_or_path, trust_remote_code=True
)


if training_args.bf16:
    dtype = torch.bfloat16
elif training_args.fp16:
    dtype = torch.float16
else:
    dtype = torch.float32

model = LlamaModelForIND.from_pretrained(
    model_args.model_name_or_path,
    torch_dtype=dtype,
    config=config,
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
).cuda()

if tokenizer.pad_token is None:
    special_token_dict["pad_token"] = DEFAULT_PAD_TOKEN
if tokenizer.eos_token is None:
    special_token_dict["eos_token"] = DEFAULT_EOS_TOKEN
if tokenizer.bos_token is None:
    special_token_dict["bos_token"] = DEFAULT_BOS_TOKEN
if tokenizer.unk_token is None:
    special_token_dict["unk_token"] = DEFAULT_UNK_TOKEN
smart_tokenizer_and_embedding_resize(
    special_tokens_dict=special_token_dict,
    tokenizer=tokenizer,
    model=model,
)
model.add_special_tokens(tokenizer)


if "Llama" in model_args.model_name_or_path or "Qwen2" in model_args.model_name_or_path:
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
elif "glm" in model_args.model_name_or_path:
    target_modules = ["query_key_value"]
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=model_args.lora_rank,
    target_modules=target_modules,  # different among different fundation model
    lora_alpha=model_args.lora_alpha,
    lora_dropout=model_args.lora_dropout,
)
model = get_peft_model(model, peft_config).cuda()
if model_args.lora_ckpt_path:  # load lora checkpoint, maybe modified
    if os.path.exists(os.path.join(model_args.lora_ckpt_path, "pytorch_model.bin")):
        paras_path = os.path.join(model_args.lora_ckpt_path, "pytorch_model.bin")
    elif os.path.exists(os.path.join(model_args.lora_ckpt_path, "adapter_model.bin")):
        paras_path = os.path.join(model_args.lora_ckpt_path, "adapter_model.bin")
    else:
        raise ValueError(
            "pytorch_model.bin or adapter_model.bin not found in the lora checkpoint"
        )
    ckpt = torch.load(paras_path)

    for k, v in model.named_parameters():
        if "lora" in k:
            if (
                "default" in k
            ):  # if using torch.save to save peft model, the key will contains "default", such as "base_model.model.model.layers.31.mlp.up_proj.default.weight"
                modify_paras_for_lora = True
            else:  # save using peftmodel.save_pretrained
                modify_paras_for_lora = False
    if modify_paras_for_lora:  # add "default" to the key of the parameters
        modified_ckpt = {}
        for k, v in ckpt.items():
            if "lora" in k and "default" not in k:
                n_list = k.split(".")
                n_list.insert(-1, "default")
                n = ".".join(n_list)
                modified_ckpt[n] = v
            else:
                modified_ckpt[k] = v
        loading_res = model.load_state_dict(modified_ckpt, strict=False)
    else:
        loading_res = model.load_state_dict(ckpt, strict=False)
    assert (
        loading_res.unexpected_keys == []
    ), f"missing keys: {loading_res.missing_keys}"
    model = model.cuda()


model.eval()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


None


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.88it/s]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaModelForIND(
      (model): LlamaModel(
        (embed_tokens): Embedding(128264, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaFlashAttention2(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features

In [3]:
raw_graph_dataset = MAGDataset(
    name="pwc_method",
    path="data/pwc/pwc_method.pickle.bin",
    raw=True,
    existing_partition=False,
)

dataset = RawDataset(
    raw_graph_dataset,
    sampling_mode=1,
    negative_size=15,
    max_pos_size=5,
    expand_factor=40,
    cache_refresh_time=64,
    test_topk=-1,
    tokenizer=tokenizer,
)

Loading terms: 2644it [00:00, 95066.86it/s]
Loading relations: 3182it [00:00, 369375.49it/s]


Partition graph ...
Finish partitioning graph ...
start saving pickle data
Save pickled dataset to data/pwc/pwc_method.pickle.bin
adding pseudo leaf
adding parent2path...
building node2pos, node2edge
building valid and test node list
1508 2113
Finish loading dataset (0.35798001289367676 seconds)


In [4]:
instruction = """
You are tasked with determining hierarchical relationships between concepts based on the given query and candidate terms. Specifically:
1. Determine if the **Hypernym Candidate** is a hypernym (broader category) of the query.
For each query and candidate pair, answer the questions with 'Yes' or 'No'.
{}
"""

local_instruct = (
    '\nQuery: "{}"'
    + '\n1. Hypernym Candidate: "{}"\n   Is this a hypernym of the query? Answer: '
    + LABEL_TOKEN
)

input_text = instruction.format(local_instruct)

In [5]:
graph = dataset.full_graph
pseudo_root_node = dataset.pseudo_root_node
root_nodes = graph.successors(pseudo_root_node)
second_level_nodes = set()
third_level_nodes = set()
# 根节点
for root in root_nodes:
    # 第二层节点
    successors = list(graph.successors(root))
    second_level_nodes.update(successors)
    # 第三层节点
    for node in successors:
        third_level_nodes.update(graph.successors(node))

test_nodes = []
for node in dataset.test_node_list:
    if node in third_level_nodes:
        test_nodes.append(node)

hit_at_1 = 0
hit_at_5 = 0
hit_at_10 = 0
recall_at_1 = 0
recall_at_5 = 0
recall_at_10 = 0

In [6]:
batch_size = 32

# second_level_nodes 分批
second_level_nodes_batches = [
    list(second_level_nodes)[i : i + batch_size]
    for i in range(0, len(second_level_nodes), batch_size)
]

for test_node in tqdm(test_nodes):
    query = test_node.description
    true_parents = set(graph.predecessors(test_node))

    parent_scores = []

    for batch in second_level_nodes_batches:

        input_texts = [input_text.format(query, node.description) for node in batch]

        inputs = tokenizer(
            input_texts,
            return_tensors="pt",
            add_special_tokens=True,
            padding="max_length",
            truncation=True,
            max_length=512,
        )

        device = "cuda" if torch.cuda.is_available() else "cpu"
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits


        YES_TOKEN_ID, NO_TOKEN_ID = tokenizer.convert_tokens_to_ids(['Yes','No'])
        LABEL_TOKEN_ID = tokenizer.convert_tokens_to_ids(LABEL_TOKEN)
        labels_pos = (inputs["input_ids"] == LABEL_TOKEN_ID).nonzero(as_tuple=False)

        for pos, parent_node in zip(labels_pos, batch):
            batch_idx, mask_idx = pos[0].item(), pos[1].item() - 1
            label_logits = logits[batch_idx, mask_idx, :]

            yes_score = label_logits[YES_TOKEN_ID].item()
            no_score = label_logits[NO_TOKEN_ID].item()

            prediction = "yes" if yes_score > no_score else "no"

            parent_scores.append(
                {
                    "parent": parent_node,
                    "yes_score": yes_score,
                    "no_score": no_score,
                    "score": yes_score - no_score,
                    "prediction": prediction,
                }
            )

 10%|█         | 23/225 [00:11<01:36,  2.10it/s]


You are tasked with determining hierarchical relationships between concepts based on the given query and candidate terms. Specifically:
1. Determine if the **Hypernym Candidate** is a hypernym (broader category) of the query.
For each query and candidate pair, answer the questions with 'Yes' or 'No'.

Query: "geglu is a specialized activation function that extends the glu mechanism by applying the gelu activation to one half of the input transformed by weight matrices \(w\) and bias \(b\), and then element-wise multiplying it with the other half transformed by \(v\) and bias \(c\). this design aims to enhance the expressiveness and performance of neural networks in various tasks."
1. Hypernym Candidate: "a bijective transformation is a mathematical function that establishes a one-to-one correspondence between two sets, ensuring that each element in the domain is uniquely paired with exactly one element in the codomain, and vice versa, thereby preserving both injectivity (no two domain

 11%|█         | 24/225 [00:11<01:36,  2.08it/s]


You are tasked with determining hierarchical relationships between concepts based on the given query and candidate terms. Specifically:
1. Determine if the **Hypernym Candidate** is a hypernym (broader category) of the query.
For each query and candidate pair, answer the questions with 'Yes' or 'No'.

Query: "geglu is a specialized activation function that extends the glu mechanism by applying the gelu activation to one half of the input transformed by weight matrices \(w\) and bias \(b\), and then element-wise multiplying it with the other half transformed by \(v\) and bias \(c\). this design aims to enhance the expressiveness and performance of neural networks in various tasks."
1. Hypernym Candidate: "trajectory data augmentation is a research method that involves systematically enhancing the quantity and diversity of trajectory datasets through the application of various algorithms and techniques, such as interpolation, extrapolation, noise injection, and synthetic data generation

 11%|█         | 25/225 [00:11<01:36,  2.08it/s]

...

 94%|█████████▍| 212/225 [01:42<00:06,  2.10it/s]


You are tasked with determining hierarchical relationships between concepts based on the given query and candidate terms. Specifically:
1. Determine if the **Hypernym Candidate** is a hypernym (broader category) of the query.
For each query and candidate pair, answer the questions with 'Yes' or 'No'.

Query: "voxel r-cnn is a two-stage 3d object detection framework that leverages voxelization for feature extraction, employs a bev region proposal network for candidate generation, and utilizes voxel roi pooling to refine object bounding boxes from 3d feature volumes. this end-to-end approach enhances detection accuracy by directly extracting and refining region-of-interest features from raw 3d point cloud data."
1. Hypernym Candidate: "a bijective transformation is a mathematical function that establishes a one-to-one correspondence between two sets, ensuring that each element in the domain is uniquely paired with exactly one element in the codomain, and vice versa, thereby preserving b

 95%|█████████▍| 213/225 [01:42<00:05,  2.10it/s]


You are tasked with determining hierarchical relationships between concepts based on the given query and candidate terms. Specifically:
1. Determine if the **Hypernym Candidate** is a hypernym (broader category) of the query.
For each query and candidate pair, answer the questions with 'Yes' or 'No'.

Query: "voxel r-cnn is a two-stage 3d object detection framework that leverages voxelization for feature extraction, employs a bev region proposal network for candidate generation, and utilizes voxel roi pooling to refine object bounding boxes from 3d feature volumes. this end-to-end approach enhances detection accuracy by directly extracting and refining region-of-interest features from raw 3d point cloud data."
1. Hypernym Candidate: "trajectory data augmentation is a research method that involves systematically enhancing the quantity and diversity of trajectory datasets through the application of various algorithms and techniques, such as interpolation, extrapolation, noise injection,

100%|██████████| 225/225 [01:48<00:00,  2.07it/s]


In [7]:
text = """You are tasked with determining hierarchical relationships between concepts based on the given query and candidate terms. Specifically:
1. Determine if the **Hypernym Candidate** is a hypernym (broader category) of the query.
For each query and candidate pair, answer the questions with 'Yes' or 'No'.

Query: "geglu is a specialized activation function that extends the glu mechanism by applying the gelu activation to one half of the input transformed by weight matrices \(w\) and bias \(b\), and then element-wise multiplying it with the other half transformed by \(v\) and bias \(c\). this design aims to enhance the expressiveness and performance of neural networks in various tasks."
1. Hypernym Candidate: "point cloud models are three-dimensional representations of objects or environments created by collecting a large set of data points in space, typically generated through techniques like lidar or 3d scanning, where each point has precise spatial coordinates (x, y, z) and often additional attributes such as color or intensity. these models are used extensively in fields like geospatial analysis, computer-aided design, and cultural heritage preservation for accurate and detailed spatial data capture and analysis."
   Is this a hypernym of the query? Answer: <label_token>"""

len(tokenizer.encode(text))

255